In [1]:
# pip install torch torchvision
import torch
import os
from os import listdir
import torchvision.transforms as T
import torch.nn as nn
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from random import randint
%matplotlib inline

In [2]:
# Import data
# https://www.kaggle.com/datasets/yashchoudhary/realsr-v3/data
# https://github.com/ngchc/CameraSR (City100)
# https://github.com/xiezw5/Component-Divide-and-Conquer-for-Real-World-Image-Super-Resolution?tab=readme-ov-file

# First, pick the 2000 image pairs and have two folders (train and test) and in each are 2 more folders (hr and lr). 
# .   
# ├── train
# │   ├── HR
# │   └── LR
# └── test
#     ├── HR
#     └── LR
# Pre-split already, 80-20 split

In [3]:
DATASET_IMAGES_DIR = "./images" # Path to your directory where the images/ folder is 
CROPPED_DATASET_IMAGES_DIR = "./cropped_images" # Path to an empty directory for the cropped images

os.makedirs("{}/train/HR".format(CROPPED_DATASET_IMAGES_DIR), exist_ok=True)
os.makedirs("{}/test/HR".format(CROPPED_DATASET_IMAGES_DIR), exist_ok=True)
os.makedirs("{}/train/LR".format(CROPPED_DATASET_IMAGES_DIR), exist_ok=True)
os.makedirs("{}/test/LR".format(CROPPED_DATASET_IMAGES_DIR), exist_ok=True)


Randomly crop the images and store them

In [4]:

directories = ["test", "train"]
for directory in directories:
	# Deal with transforming both the LR and HR images at the same time in order 
	# to transform them the same way
	HR_dir = sorted(os.listdir("{}/{}/HR".format(DATASET_IMAGES_DIR, directory)))
	LR_dir = sorted(os.listdir("{}/{}/LR".format(DATASET_IMAGES_DIR, directory)))
	for imageIndex,img in enumerate(LR_dir):

		imgLR = Image.open("{}/{}/LR/".format(DATASET_IMAGES_DIR, directory)+LR_dir[imageIndex])
		imgHR = Image.open("{}/{}/HR/".format(DATASET_IMAGES_DIR, directory)+HR_dir[imageIndex])

		# need to handle separately since some image pairs have same sizes while some are x3
		crop_size = 64
		if imgLR.size == imgHR.size:  # same size, bicubic downsample
			w, h = imgLR.size
			assert w >= crop_size and h >= crop_size
			x0 = randint(0, w-3*crop_size-1)
			y0 = randint(0, h-3*crop_size-1)
			imgLR = imgLR.crop((x0, y0, x0+3*crop_size, y0+3*crop_size))
			imgHR = imgHR.crop((x0, y0, x0+3*crop_size, y0+3*crop_size))
			imgLR = imgLR.resize((crop_size, crop_size), Image.BICUBIC)
		else:   # x3
			w, h = imgLR.size
			assert w >= crop_size and h >= crop_size
			assert imgHR.size == (3*w, 3*h)
			x0 = randint(0, w-crop_size-1)
			y0 = randint(0, h-crop_size-1)
			imgLR = imgLR.crop((x0, y0, x0+crop_size, y0+crop_size))
			imgHR = imgHR.crop((3*x0, 3*y0, 3*x0+3*crop_size, 3*y0+3*crop_size))

		assert imgLR.size == (crop_size, crop_size)
		assert imgHR.size == (3*crop_size, 3*crop_size)

		# Save the images in the cropped dataset folder
		imgLR.save("{}/{}/LR/".format(CROPPED_DATASET_IMAGES_DIR, directory)+LR_dir[imageIndex])
		imgHR.save("{}/{}/HR/".format(CROPPED_DATASET_IMAGES_DIR, directory)+HR_dir[imageIndex])